In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps
import math
import calendar
from datetime import date
from edc_base.utils import get_utcnow

edc_protocol = django_apps.get_app_config('edc_protocol')

In [2]:
def get_holidays(year, month):
    facility_app_config = django_apps.get_app_config('edc_facility')

    facility = facility_app_config.get_facility('5-day clinic')

    holiday_list = facility.holidays.holidays.filter(
        local_date__year=year,
        local_date__month=month).values_list('local_date', flat=True)
    return [h.day for h in holiday_list]

In [3]:
def get_scheduled_pids():
    
    return ParticipantNote.objects.filter(
        title = 'Follow Up').values_list('subject_identifier', flat=True)


In [4]:
def get_last_day_scheduled(year, month):
    
    last_part_notes = ParticipantNote.objects.filter(title = 'Follow Up',
                                                     date__year=year,
                                                     date__month=month)
    if last_part_notes:
        return last_part_notes[0].date.day, last_part_notes.count()
    return None, 0



In [5]:
def get_dates_to_schedule(year, month, last_day_scheduled=None, full_scheduled=None):
    calendar_obj = calendar.Calendar(firstweekday=0)
    
    dates_to_schedule = []
    count = 0
    for month_day in calendar_obj.itermonthdays(year, month):
        
        if last_day_scheduled and not full_scheduled and count == 0:
            dates_to_schedule.append(date(year, month, last_day_scheduled))
            count += 1
            
        last_day_scheduled = last_day_scheduled or 0
        if (month_day != 0 and month_day > last_day_scheduled and 
            calendar.weekday(year, month, month_day) not in [5,6] and
            month_day not in get_holidays(year, month)):
            
            dates_to_schedule.append(date(year, month, month_day))
                
    return dates_to_schedule


In [9]:
def get_consents_for_the_month(year, month, days_available, cohort_pids):
    
    cohart_a_pids = OnScheduleChildCohortAEnrollment.objects.values_list(
    'subject_identifier', flat=True).distinct()
    
    child_consents = CaregiverChildConsent.objects.filter(
        Q(child_dob__month=6) & Q(child_dob__year=2017),
        child_dob__isnull=False, subject_identifier__in=cohart_a_pids,).exclude(
        subject_identifier__in=get_scheduled_pids()).order_by('child_dob')[:days_available*2]
    
    return child_consents

In [70]:
from dateutil.relativedelta import relativedelta

def schedule_fu_visit(year, month, max_per_day, cohort_pids):
    
    last_day_scheduled, last_daycount = get_last_day_scheduled(year, month)
    if ((not last_day_scheduled and month == get_utcnow().date().month and year == get_utcnow().date().year)
        or date(year, month, last_day_scheduled) >= get_utcnow()):
        last_day_scheduled = (get_utcnow()+relativedelta(days=1)).date().day
    
    days_available = get_dates_to_schedule(
        year, month,
        last_day_scheduled=last_day_scheduled,
        full_scheduled=last_daycount==max_per_day)
    
    consents_for_month = get_consents_for_the_month(year, month, len(days_available), cohort_pids)
    print(consents_for_month.count())
    
    last_mod = last_daycount % max_per_day
    
    for i in range(len(days_available)):
        upper_limit = ((i+1)*2) + last_mod
        lower_limit = i*2 + last_mod
        
        schedule_date = (days_available[i])
        print(schedule_date)
        
        sliced_consents = consents_for_month[lower_limit:upper_limit]
        for consent in sliced_consents:
#             print(lower_limit, "??????????", upper_limit)
            
            print(sliced_consents.count(), ">>>>>>>", consent)
        try:
            ParticipantNote.objects.get(subject_identifier=consent.subject_identifier)
        except ParticipantNote.DoesNotExist:
            ParticipantNote.objects.create(
                subject_identifier = consent.subject_identifier,
                title = 'Follow Up',
                color = 'yellow',
                date = schedule_date,
                description = consent.child_dob)


In [71]:
ParticipantNote.objects.filter(date__year=2022, date__month=7)

<QuerySet []>

In [72]:
cohart_a_pids = OnScheduleChildCohortAEnrollment.objects.values_list('subject_identifier', flat=True).distinct()

schedule_fu_visit(2022, 7, 2, cohart_a_pids)

6
2022-07-23
2 >>>>>>> CaregiverChildConsent object (950c210b-2dc7-4f6d-8608-d59e589aa80a)
2 >>>>>>> CaregiverChildConsent object (9bdd93aa-e383-4100-8c9b-acf1b561a4d1)
2022-07-25
2 >>>>>>> CaregiverChildConsent object (7ef61f38-7a15-4bfa-bdb0-05ce307e3f42)
2 >>>>>>> CaregiverChildConsent object (be50ebeb-61da-404b-bee3-9b0b10b10626)
2022-07-26
2022-07-27
2022-07-28
2022-07-29


In [51]:
ParticipantNote.objects.filter(date__year=2022, date__month=7).delete()

(4, {'flourish_calendar.ParticipantNote': 4})

In [12]:
get_utcnow().date().day

22

In [ ]:
##########################################

In [33]:
child_consents6 = CaregiverChildConsent.objects.filter(
    Q(child_dob__month=6) & Q(child_dob__year=2017),child_dob__isnull=False, subject_identifier__in=list(cohart_a_pids) ).order_by('child_dob')

for a in child_consents6:
    print(a.subject_identifier, a.child_dob, a.consent_datetime.date())

B142-040990584-0-10 2017-06-05 2022-01-24
B142-040990444-7-10 2017-06-17 2021-10-27
B142-040990735-8-10 2017-06-19 2022-04-08
B142-040990657-4-10 2017-06-20 2022-02-22
B142-040990467-8-10 2017-06-20 2021-11-04
B142-040990450-4-10 2017-06-26 2021-10-29


In [28]:
child_consents6.count()

6

In [26]:
for a in child_consents7:
    print(a.subject_identifier)

B142-040990456-1-10
B142-040990406-6-10
B142-040990541-0-10
B142-040990566-7-10
B142-040990395-1-10
B142-040990588-1-10
B142-040990498-3-10


In [34]:
child_consents7 = CaregiverChildConsent.objects.filter(
    Q(child_dob__month__in=[6,7]) & Q(child_dob__year=2017),child_dob__isnull=False, subject_identifier__in=list(cohart_a_pids) ).order_by('child_dob')

for a in child_consents7:
    print(a.subject_identifier, a.child_dob, a.consent_datetime.date())

B142-040990456-1-10 2017-07-05 2021-11-01
B142-040990406-6-10 2017-07-11 2021-10-15
B142-040990541-0-10 2017-07-20 2021-12-16
B142-040990566-7-10 2017-07-20 2022-01-19
B142-040990395-1-10 2017-07-21 2021-10-12
B142-040990588-1-10 2017-07-23 2022-01-25
B142-040990498-3-10 2017-07-27 2021-11-19


In [59]:
child_consents7[6:8]

[<CaregiverChildConsent: CaregiverChildConsent object (f528a730-1377-4ced-afa2-65f738ed9ba0)>]

In [ ]:
iterations = int(math.ceil(float(x)/2))


for x in range(iterations):
    y = (x+1)*2
    for consent in child_consents7[x*2:y]:
        ParticipantNote.objects.create(
            subject_identifier = consent.subject_identifier,
            title = 'Follow Up',
            color = 'yellow',
            date = schedule_date,
            description = consent.child_dob
        )
        

6